In [1]:
import numpy as np
import pandas as pd
import folium
import json
from geopy.geocoders import Nominatim
import requests
import re
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import time
import math
%matplotlib inline

### Data Scraping from Web for Chennai City

In [2]:
data_url="https://en.wikipedia.org/wiki/Areas_of_Chennai"

In [3]:
import requests 
from urllib.request import urlopen
from bs4 import BeautifulSoup 


r = requests.get(data_url) 
html=urlopen(data_url)
soup = BeautifulSoup(html,'html.parser')
My_table = soup.find('table',{'class':'wikitable'})
data_table = My_table.findAll('td')
name=[]
for i,data in enumerate(data_table):
    name.append(data.text.strip())

My_table = soup.find('table',{'class':'wikitable'})
name=[]
latitude=[]
longitude=[]
data_table=My_table.findAll('tr',attrs={'class':'tab'})
for i,data in enumerate(data_table):
    if i!=105:
        row=data.find_all('td')
        name.append(row[1].text.strip())
        latitude.append(row[2].text.strip())
        longitude.append(row[3].text.strip())
data=pd.DataFrame({"S.No":range(1,len(name)+1),
                   "Location":name,
                   "Latitude":latitude,
                   "Longitude":longitude
                  })

### Obtain Latitude and Longitude

In [4]:
len(name)

163

In [5]:
name

['Adambakkam',
 'Adyar',
 'Alandur',
 'Alapakkam',
 'Alwarpet',
 'Alwarthirunagar',
 'Ambattur',
 'Aminjikarai',
 'Anna Nagar',
 'Annanur',
 'Arumbakkam',
 'Ashok Nagar',
 'Avadi',
 'Ayanavaram',
 'Besant Nagar',
 'Basin Bridge',
 'Chepauk',
 'Chetput',
 'Chintadripet',
 'Chitlapakkam',
 'Choolai',
 'Choolaimedu',
 'Chrompet',
 'Egmore',
 'Ekkaduthangal',
 'Eranavur',
 'Ennore',
 'Foreshore Estate',
 'Fort St. George',
 'George Town',
 'Gopalapuram',
 'Government Estate',
 'Guindy',
 'Gerugambakkam',
 'IIT Madras',
 'Injambakkam',
 'ICF',
 'Iyyapanthangal',
 'Jafferkhanpet',
 'Karapakkam',
 'Kattivakkam',
 'Kattupakkam',
 'Kazhipattur',
 'K.K. Nagar',
 'Keelkattalai',
 'Kattivakkam',
 'Kilpauk',
 'Kodambakkam',
 'Kodungaiyur',
 'Kolathur',
 'Korattur',
 'Korukkupet',
 'Kottivakkam',
 'Kotturpuram',
 'Kottur',
 'Kovilambakkam',
 'Koyambedu',
 'Kundrathur',
 'Madhavaram',
 'Madhavaram Milk Colony',
 'Madipakkam',
 'Madambakkam',
 'Maduravoyal',
 'Manali',
 'Manali New Town',
 'Manapakkam

In [6]:
def lat_lon(x):
    try:
        address=x
        geolocator=Nominatim(user_agent="My application")
        location=geolocator.geocode(address)
        return location.latitude,location.longitude
    except:
        return 0,0


In [7]:
latitude=[]
longitude=[]

In [8]:
for i in range(0,20):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)


In [9]:
latitude

[12.9822215,
 13.00645,
 13.00282155,
 13.049901,
 13.0338602,
 13.047486,
 13.1128863,
 13.0721399,
 0,
 13.1163563,
 13.074371,
 13.0400731,
 13.1254758,
 0,
 12.9996907,
 13.1080471,
 13.06363,
 12.4675265,
 13.0761258,
 12.9330984]

In [10]:
for i in range(20,40):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)
for i in range(40,60):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)
for i in range(60,80):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)
for i in range(80,100):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)
for i in range(100,120):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)
for i in range(120,140):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)
for i in range(140,len(name)):
    lat,lon=lat_lon(name[i])
    latitude.append(lat)
    longitude.append(lon)





In [11]:
print(len(latitude),len(longitude))

163 163


In [12]:
data=pd.DataFrame({"Location":name,"Latitude":latitude,"Longitude":longitude})

In [13]:
data=data.append({"Location":"Velappanchavadi","Latitude":13.05849,"Longitude":80.134405,"Cluster":0},ignore_index=True,)

In [14]:
data=data.append({"Location":"Ekkaduthangal","Latitude":13.02248,"Longitude":80.203187},ignore_index=True)
data=data.append({"Location":"Eranavur","Latitude":13.189569,"Longitude":80.3039},ignore_index=True)
data=data.append({"Location":"Kattivakkam","Latitude":13.216133,"Longitude":80.318177},ignore_index=True)
data=data.append({"Location":"Kazhipattur","Latitude":12.823581,"Longitude":80.231043},ignore_index=True)
data=data.append({"Location":"Perumbakkam","Latitude":12.892316,"Longitude":80.188892},ignore_index=True)
data=data.append({"Location":"Pudupet","Latitude":12.532496,"Longitude":78.510064},ignore_index=True)
data=data.append({"Location":"Pulianthope","Latitude":13.089714,"Longitude":80.254126},ignore_index=True)
data=data.append({"Location":"Purasaiwalkam","Latitude":13.02248,"Longitude":80.203187},ignore_index=True)
data=data.append({"Location":"Puthagaram","Latitude":10.461796,"Longitude":79.523907},ignore_index=True)
data=data.append({"Location":"Surapet","Latitude":13.145407,"Longitude":80.183808},ignore_index=True)
data=data.append({"Location":"Tiruverkadu","Latitude":13.070637,"Longitude":80.128979},ignore_index=True)
data=data.append({"Location":"Thiruvotriyur","Latitude":13.1691914,"Longitude":80.3045793},ignore_index=True)
data=data.append({"Location":"Thiruvallikeni","Latitude":13.0587107,"Longitude":80.27570630000002},ignore_index=True)
data=data.append({"Location":"Velappanchavadi","Latitude":13.058523,"Longitude":80.135452},ignore_index=True)



In [15]:
data[data['Location']=='ICF']

,Location,Latitude,Longitude,Cluster
36,ICF,44.277012,-105.508473,NaN


In [16]:
data.loc[data['Location']=='Chetput','Latitude']=13.071521
data.loc[data['Location']=='Chetput','Longitude']=80.241516
data.loc[data['Location']=='Chetput']

,Location,Latitude,Longitude,Cluster
17,Chetput,13.071521,80.241516,NaN


In [17]:
waste=data[data['Latitude']==0].index

In [18]:
waste

Int64Index([  8,  13,  24,  25,  40,  42,  45,  65,  75,  76, 111, 114, 124,
            134, 136, 144, 145, 148, 157],
           dtype='int64')

In [19]:
data=data.drop(waste,axis=0)

In [20]:
data=data.drop(162)

In [21]:
data[data['Latitude']<10]

,Location,Latitude,Longitude,Cluster
29,George Town,5.414568,100.329803,NaN
68,Mannady,9.088030,76.737411,NaN
139,Thirumangalam,9.798009,77.929843,NaN


In [22]:
data.sort_values("Latitude",inplace=True)

In [23]:
data.index=range(len(data))

In [24]:
data.head()

,Location,Latitude,Longitude,Cluster
0,George Town,5.414568,100.329803,NaN
1,Mannady,9.088030,76.737411,NaN
2,Thirumangalam,9.798009,77.929843,NaN
3,Puthagaram,10.461796,79.523907,NaN
4,K.K. Nagar,11.210465,78.169173,NaN


In [25]:
data.index=range(0,len(data))

In [26]:
data

,Location,Latitude,Longitude,Cluster
0,George Town,5.414568,100.329803,NaN
1,Mannady,9.088030,76.737411,NaN
2,Thirumangalam,9.798009,77.929843,NaN
3,Puthagaram,10.461796,79.523907,NaN
4,K.K. Nagar,11.210465,78.169173,NaN
...,...,...,...,...
153,Raj Bhavan,17.419422,78.460455,NaN
154,Ramavaram,17.770062,81.864590,NaN
155,Manali New Town,32.245461,77.187293,NaN
156,Manali,32.263094,77.188122,NaN


In [27]:
data.to_csv("chennai_coordinates.csv")

# Marking All locations in Map

In [ ]:
address='Chennai'

geolocator=Nominatim()
location=geolocator.geocode(address)
che_lat=location.latitude
che_lon=location.longitude

In [ ]:
che_lat

In [ ]:
def plot_map(data,clusters):

    map_chennai=folium.Map(location=[che_lat,che_lon],zoom_start=10)
    colors = ['red','blue','white','yellow']
    for lat,lng,loc,cluster in zip(data['Latitude'],data['Longitude'],data['Location'],data['Cluster']):
        label='{}'.format(loc)
        label=folium.Popup(label,parse_html=True)
        folium.CircleMarker(
        location=[lat,lng],
        radius=5,
        color='black',
        popup=label,
        fill=True,
        fill_color=colors[int(cluster%4)],
        fill_opacity=0.7,
        ).add_to(map_chennai)
    return map_chennai

In [ ]:
map_chennai=folium.Map(location=[che_lat,che_lon],zoom_start=10)
#color=['red','blue','yellow']
for lat,lng,loc in zip(data['Latitude'],data['Longitude'],data['Location']):
    label='{}-{},{}'.format(loc,lat,lng)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    location=[lat,lng],
    radius=5,
    color='red',
    popup=label,
    fill=True,
    fill_opacity=1,
    parse_html=True
    ).add_to(map_chennai)
    
map_chennai

### Choosing Best 'K' Value

In [ ]:
chennai_cluster=data.drop('Location',axis=1)
inertia=[]
distortion=[]
for i in range(1,11):
    kmeans=KMeans(n_clusters=i,random_state=0).fit(chennai_cluster)
    inertia.append(kmeans.inertia_)
    distortion.append(sum(np.min(cdist(chennai_cluster, kmeans.cluster_centers_, 
                 'euclidean'),axis=1)) / chennai_cluster.shape[0])
    
    

In [ ]:
fig=plt.Figure(figsize=(10,5))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)
ax1.plot(range(1,11),inertia)
ax2.plot(range(1,11),distortion)
ax1.set_title('Elbow using Inertia')
ax2.set_title('Elbow using Distortion')
ax1.set_xlabel('K')
ax1.set_ylabel('Inertia')
ax2.set_xlabel('K')
ax2.set_ylabel('Distortion')
plt.tight_layout("best")
fig

## Plotting Chennai Map based on cluster

### Clustering the Map

In [ ]:
data.columns.shape[0]

In [ ]:
n_clusters=4
if(data.columns.shape[0]>3):
    kmeans=KMeans(n_clusters,random_state=5,max_iter=3000).fit(data.drop(['Location','Distance','Cluster'],axis=1))
else:
    kmeans=KMeans(n_clusters,random_state=5,max_iter=3000).fit(data.drop(['Location'],axis=1))
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
data['Cluster']=kmeans.labels_

In [ ]:
def compute_dist(lat,lon,cluster):
    diff_1=kmeans.cluster_centers_[cluster][0]-lat
    diff_2=kmeans.cluster_centers_[cluster][1]-lon
    #print(diff_2**2+diff_1**2)
    return math.sqrt(diff_1**2+diff_2**2)

In [ ]:
print(compute_dist(1,2,2))

In [ ]:
data['Distance']=0#chennai.apply(compute_dist(chennai['Latitude'],chennai['Longitude'],chennai['Cluster']))

In [ ]:
for i in range(len(chennai)):
    data['Distance'][i]=compute_dist(data['Latitude'][i],data['Longitude'][i],data['Cluster'][i])

In [ ]:
plt.figure(figsize=(20,15))
color=['red','blue','green','black']
for i in range(0,kmeans.cluster_centers_.shape[0]):
    plt.scatter(chennai[chennai['Cluster']==i]['Longitude'],chennai[chennai['Cluster']==i]['Latitude'],c=color[i],s=100)
plt.scatter(kmeans.cluster_centers_[:,1],kmeans.cluster_centers_[:,0],c='yellow',s=300)
for i in range(0,len(data)):
     plt.text(chennai['Longitude'][i],chennai['Latitude'][i],s=chennai['Location'][i],size=10)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
plot_map(data,n_clusters)

In [ ]:
#data.to_csv("Chennai_Clustered.csv")